In [ ]:
import os
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import configparser

# Fetch login page

In [ ]:
s = requests.Session()

In [ ]:
r = s.get('https://selvbetjening.rejsekort.dk/CWS/Home/UserNameLogin')

In [ ]:
r.status_code

# Extract `RequestVerificationToken` from login page

In [ ]:
m = re.search(pattern='<input name="__RequestVerificationToken".*', string=r.text)

In [ ]:
soup = BeautifulSoup(m.group(0), 'lxml')

In [ ]:
RequestVerificationToken = soup.findAll('input')[0]['value']

In [ ]:
RequestVerificationToken

# Log in

In [ ]:
config = configparser.ConfigParser()
config.read('creds.ini')

In [ ]:
login_url = 'https://selvbetjening.rejsekort.dk/CWS/Home/Index'

In [ ]:
rr = s.post(login_url, data={'Username': config['creds']['username'],
                             'Password': config['creds']['password'],
                             '__RequestVerificationToken': RequestVerificationToken})

In [ ]:
rr.status_code

In [ ]:
if not os.path.exists('tmp'):
    os.makedirs('tmp')

In [ ]:
with open('tmp/login.html', 'w') as f:
    f.write(rr.text)

# Fetch list of recent travels

In [ ]:
recent_travels = s.get('https://selvbetjening.rejsekort.dk/CWS/TransactionServices/TravelCardHistory')

In [ ]:
recent_travels.status_code

In [ ]:
with open('tmp/recent_travels.html', 'w') as f:
    f.write(recent_travels.text)

Get new token for fetching all travels

In [ ]:
m = re.search(pattern='var antiForgeryToken = \'<input name="__RequestVerificationToken".*', string=recent_travels.text)

In [ ]:
soup = BeautifulSoup(m.group(0), 'lxml')

In [ ]:
RequestVerificationToken = soup.findAll('input')[0]['value']

In [ ]:
RequestVerificationToken

# Fetch all travels

In [ ]:
travels = s.post('https://selvbetjening.rejsekort.dk/CWS/TransactionServices/TravelCardHistory',
                 data={'periodSelected': 'All',
                       '__RequestVerificationToken': RequestVerificationToken})

In [ ]:
travels.status_code

In [ ]:
with open('tmp/travels.html', 'w') as f:
    f.write(travels.text)

TODO

- Extract data for all travels and populate dataframe

In [ ]:
soup = BeautifulSoup(travels.text, 'lxml')

One html table for each page of travels

In [ ]:
travels_tables = soup.findAll('table')

In [ ]:
len(travels_tables)

https://srome.github.io/Parsing-HTML-Tables-in-Python-with-BeautifulSoup-and-pandas/